In [1]:
#autoreload
%load_ext autoreload
%autoreload 2

import torch
from torchmetrics import AUROC
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import json
import os

import sys
sys.path.append("../../src_clasificacion_vistas/common")

import pycimg

from pathlib import Path
import cv2
from tqdm import tqdm

In [2]:
current_file_dir = os.getcwd()
train_results_json="../out_evaluate_repaso_insect2_gamma_0/train_scores.json"
val_results_json="../out_evaluate_repaso_insect2_gamma_0/val_scores.json"
train_results_json=os.path.join(current_file_dir,train_results_json)    
val_results_json=os.path.join(current_file_dir,val_results_json)    
out_dir="/home/aalbiol/Desktop/repasoC"

Path(out_dir).mkdir(parents=True, exist_ok=True)


def create_lists(train_data,clases):
    listas_positivos={} # Casos po
    listas_negativos={}
    for defecto in clases:
        listas_positivos[defecto]=[]
        listas_negativos[defecto]=[]
    for d in tqdm(train_data):

        filenamejson=d['filename'].replace(".npz",".json")
        scores=d['probs_vistas']
        kk=os.path.join("..",filenamejson)
        npzfilename=kk.replace(".json",".npz")
#        imagenes=pycimg.npzread_torch(npzfilename,kk,channel_list=['R','G','B'])
        if not os.path.exists(kk):
            print(" does not exist: ",kk)
            continue
        with open(kk, "r") as f:
            anot_dict = json.load(f)
        if 'views_annotations' not in anot_dict:
            continue    
        truth_vistas=anot_dict['views_annotations']
        
        nvistas=len(truth_vistas)
        nclases=len(clases)
        truths=[]
        
        for n in range(nvistas):
            keyid="view_"+str(n)
            truths.append(truth_vistas[keyid])

            
        gt=torch.zeros((nvistas,nclases))    
        
        
        # truths: lista de vistas. Cada una de ellas es un diccionario con el nombre del defecto como clave
        for n in range(nvistas):
            for m, defecto in enumerate(clases):
                if defecto in truths[n]:
                    gt[n,m]=float(truths[n][defecto])
                    
        preds1=torch.zeros((nvistas,nclases))
        for n in range(nvistas):
            for m, defecto in enumerate(clases):
                if defecto in scores:
                    preds1[n,m]=float(scores[defecto][n])
                    if gt[n,m]==1:
                        listas_positivos[defecto].append((filenamejson,n,preds1[n,m].item()))
                    else:
                        listas_negativos[defecto].append((filenamejson,n,preds1[n,m].item()))
        
                    
        # gt de este json
        # preds1 de este json                    




    return listas_positivos,listas_negativos





In [3]:
## Leer los archivos

with open(train_results_json, "r") as f:
    train_dict = json.load(f)

with open(val_results_json, "r") as f:
    val_dict = json.load(f)    

train_data=train_dict['train_results']
val_data = val_dict['val_results']
clases=list(train_dict['train_results'][0]['ground_truth'].keys())

all_data=train_data+val_data

print(len(train_data))
print(len(val_data))
print(len(all_data))

1318
351
1669


In [4]:
listas_positivas_all, listas_negativas_all=create_lists(all_data,clases)

  0%|          | 0/1669 [00:00<?, ?it/s]

100%|██████████| 1669/1669 [00:02<00:00, 625.50it/s]


In [5]:
listas_negativas_all['cracks'][:3]

[('data/mandarins/clementine/KDC/20220808_154231_Img_20220808_154156produccionClementina/20250218_1154_000579.json',
  0,
  0.012000000104308128),
 ('data/mandarins/clementine/KDC/20220808_154231_Img_20220808_154156produccionClementina/20250218_1154_000579.json',
  1,
  0.012000000104308128),
 ('data/mandarins/clementine/KDC/20220808_154231_Img_20220808_154156produccionClementina/20250218_1154_000579.json',
  2,
  0.008999999612569809)]

# Cantidad de Vistas de con cada defecto

In [6]:
for c in clases:
    print(c)
    print("positivos: ",len(listas_positivas_all[c]))
    print("negativos: ",len(listas_negativas_all[c]))
    print("total: ",len(listas_positivas_all[c])+len(listas_negativas_all[c]))
    print(" ")

cracks
positivos:  202
negativos:  15781
total:  15983
 
damage_by_pliers
positivos:  159
negativos:  15824
total:  15983
 
deformed_peduncle
positivos:  539
negativos:  15444
total:  15983
 
green
positivos:  733
negativos:  15250
total:  15983
 
hollow
positivos:  357
negativos:  15626
total:  15983
 
insect
positivos:  126
negativos:  15857
total:  15983
 
light_scars
positivos:  833
negativos:  15150
total:  15983
 
mechanical_damage
positivos:  208
negativos:  15775
total:  15983
 
oil_spots
positivos:  327
negativos:  15656
total:  15983
 
plu_stickers
positivos:  564
negativos:  15419
total:  15983
 
reddish
positivos:  280
negativos:  15703
total:  15983
 
rotten
positivos:  660
negativos:  15323
total:  15983
 
scars
positivos:  768
negativos:  15215
total:  15983
 
skin_breakdown
positivos:  507
negativos:  15476
total:  15983
 
surface
positivos:  1730
negativos:  14253
total:  15983
 


# Ordenar listas de por scores
Si hay más de 500

* Los positivos seleccionamos los 500 de menos score
* Los negativos sleccionamos los 500 de más score

In [7]:
seleccion_positivos={}
seleccion_negativos={}
for defecto in clases:
    seleccion_positivos[defecto]=[]
    seleccion_negativos[defecto]=[]
NMAX=200
for defecto in clases:
    if len(listas_positivas_all[defecto])<NMAX:
        
        seleccion_positivos[defecto]=listas_positivas_all[defecto]
    else:
        lp=listas_positivas_all[defecto]

        scores=[x[2] for x in lp]
        scores=np.array(scores)
        idx_sorted = np.argsort(scores)
        idx_sorted = idx_sorted[:NMAX]
        for idx in idx_sorted:
            seleccion_positivos[defecto].append(lp[idx])
    ln=listas_negativas_all[defecto]            
    scores=[x[2] for x in ln]
    scores=np.array(scores)
    idx_sorted = np.argsort(scores)[::-1]
    idx_sorted = idx_sorted[:NMAX]
    for idx in idx_sorted:
        seleccion_negativos[defecto].append(ln[idx])


In [8]:
for c in clases:
    print(c)
    print("positivos: ",len(seleccion_positivos[c]), " negativos: ",len(seleccion_negativos[c]))
    #print("negativos: ",len(listas_negativas_all[c]))
    #print("total: ",len(listas_positivas_all[c])+len(listas_negativas_all[c]))
    print(" ")

cracks
positivos:  200  negativos:  200
 
damage_by_pliers
positivos:  159  negativos:  200
 
deformed_peduncle
positivos:  200  negativos:  200
 
green
positivos:  200  negativos:  200
 
hollow
positivos:  200  negativos:  200
 
insect
positivos:  126  negativos:  200
 
light_scars
positivos:  200  negativos:  200
 
mechanical_damage
positivos:  200  negativos:  200
 
oil_spots
positivos:  200  negativos:  200
 
plu_stickers
positivos:  200  negativos:  200
 
reddish
positivos:  200  negativos:  200
 
rotten
positivos:  200  negativos:  200
 
scars
positivos:  200  negativos:  200
 
skin_breakdown
positivos:  200  negativos:  200
 
surface
positivos:  200  negativos:  200
 


# Crear imágenes de vistas

In [9]:
from tqdm import tqdm
for defecto in clases:
    print(">>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>")
    print(defecto,len(seleccion_positivos[defecto]),len(seleccion_negativos[defecto]))
    lista_positivos=seleccion_positivos[defecto]
    lista_negativos=seleccion_negativos[defecto]
    
    # Crear un directorio para cada defecto
    
    positive_dir=out_dir+"/positive/"+defecto
    negative_dir=out_dir+"/negative/"+defecto
    Path(positive_dir).mkdir(parents=True, exist_ok=True)
    Path(negative_dir).mkdir(parents=True, exist_ok=True)
    print(" >>> Positive")
    for vista in tqdm(lista_positivos):
        json_file,view_number,score=vista
        npzfilename=json_file.replace(".json",".npz")
        npzfilename=os.path.join("..",npzfilename)
        json_file=os.path.join("..",json_file)
        imagenes=pycimg.npzread_torch(npzfilename,json_file,channel_list=['R','G','B'])
        imagen=imagenes[view_number].numpy()
        imagen=imagen.transpose((1,2,0))
        imagen=(255*imagen).astype(np.uint8)
        imagen_bgr=imagen[:,:,::-1]
        imagen_bgr=cv2.resize(imagen_bgr,(224,224))
        #print(f'Vista positiva: {json_file} {view_number} {imagen.shape} {imagen.dtype} {imagen.min()} {imagen.max()}')
        
        basename=os.path.basename(json_file)
        basename=basename.replace(".json","")
        score_int=int(score*1000)
        basename=f'{basename}_{view_number:02d}_{score_int:02d}.jpg'
        #print(f'Vista positiva: {basename} {view_number} {imagen.shape} {imagen.dtype} {imagen.min()} {imagen.max()}')
        outname=os.path.join(positive_dir,basename)
        cv2.imwrite(outname,imagen_bgr)
    print(" >>> Negative")        
    for vista in tqdm(lista_negativos):
        json_file,view_number,score=vista
        npzfilename=json_file.replace(".json",".npz")
        npzfilename=os.path.join("..",npzfilename)
        json_file=os.path.join("..",json_file)        
        imagenes=pycimg.npzread_torch(npzfilename,json_file,channel_list=['R','G','B'])
        imagen=imagenes[view_number].numpy()
        imagen=imagen.transpose((1,2,0))
        imagen=(255*imagen).astype(np.uint8)
        imagen_bgr=imagen[:,:,::-1]
        imagen_bgr=cv2.resize(imagen_bgr,(224,224))
        #print(f'Vista positiva: {json_file} {view_number} {imagen.shape} {imagen.dtype} {imagen.min()} {imagen.max()}')
        
        basename=os.path.basename(json_file)
        basename=basename.replace(".json","")
        score_int=int(score*1000)        
        basename=f'{basename}_{view_number:02d}_{score_int:02d}.jpg'
        #print(f'Vista positiva: {basename} {view_number} {imagen.shape} {imagen.dtype} {imagen.min()} {imagen.max()}')
        outname=os.path.join(negative_dir,basename)
        #print(outname)
        cv2.imwrite(outname,imagen_bgr)        

>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
cracks 200 200
 >>> Positive


  0%|          | 0/200 [00:00<?, ?it/s]

100%|██████████| 200/200 [00:01<00:00, 178.18it/s]


 >>> Negative


100%|██████████| 200/200 [00:01<00:00, 158.09it/s]


...

100%|██████████| 200/200 [00:01<00:00, 160.65it/s]


>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
surface 200 200
 >>> Positive


100%|██████████| 200/200 [00:01<00:00, 137.62it/s]


 >>> Negative


100%|██████████| 200/200 [00:01<00:00, 148.01it/s]
